In [37]:
import re
from pathlib import Path
from datetime import datetime
import time
import os
from tqdm import tqdm
from itertools import groupby
from operator import itemgetter

In [48]:
test = [("1", "hello")] * 3 + [("2", "bye")] * 3 + [("1", "hello")] * 3 + [("2", "bye")] * 3
for i, group in groupby(test, key = itemgetter(0)):
    for j, text in group:
        print(text)


hello
hello
hello
bye
bye
bye
hello
hello
hello
bye
bye
bye


In [53]:
def prepare_whatsapp_data(directory):
    def get_date_from_msg(msg):
        date = re.findall(r"(\d+/\d+/\d+, \d+:\d+) -", msg)

        if date:
            return time.mktime(datetime.strptime(date[0], r"%m/%d/%y, %H:%M").timetuple())
        else:
            return 0
        
    def get_sorted_sequences(filename):
        f = open(filename, "r", encoding = "utf-8")
        messages = []
        index = 0
        for line in f:
            if re.match(r"\d+/\d+/\d+, \d+:\d+", line):
                messages.append(line)
                index += 1
            else:
                messages[index - 1] += line

        messages.sort(key = get_date_from_msg)

        return messages
        
    for filename in Path(directory).glob("*.txt"):
        # get head and tail of path
        split = os.path.split(filename)
        # create new "edited" directory
        path = os.path.join(split[0], "edited")
        # create directory if not existant
        if not os.path.exists(path):
            os.mkdir(path)
        # start a new file for the current whatsapp chat
        with open(os.path.join(split[0], "edited", split[1]), "w+", encoding = "utf-8") as f:
            # get list of messages sorted by time
            messages = get_sorted_sequences(filename)
            # get list of tuples of author and message for every message
            messages = [(author, text.strip()) for msg in messages for author, text in re.findall(r".+? - (.+?): (.+)", msg, flags = re.DOTALL)]
            # group consecutive messages of the same author
            groups = groupby(messages, key = itemgetter(0))
            # get list of grouped messages where consecutive texts of one author are separated by "<new>""
            messages = [" <new> ".join([msg for author, msg in group]) for i, group in groups]
            
            for i in range(len(messages) - 1):
                f.write(messages[i] + "\t" + messages[i + 1] + "\n")
            

In [27]:
re.findall("(he)", "hehe")

['he', 'he']

In [52]:
prepare_whatsapp_data("chats")